In [2]:
%run useful_functions.py

C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:951: SyntaxWarning: invalid escape sequence '\s'
  item1A = re.sub('\s{2,}',' ', item1A)
C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:962: SyntaxWarning: invalid escape sequence '\s'
  document = re.sub('\s{2,}',' ', document)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


update 22


In [2]:
import wrds

In [3]:
# Idea: we download and create for each stock a dataframe containing the data, 
# then, at the end, we put them all together to have a nice dataframe for each firms

#read in the previously saved stocknames file
stocknames = pd.read_csv("../../data/stocknames.csv.gz", na_filter = False)
stocknames.replace('', np.nan, inplace = True)

In [4]:
#create a empty file for each ticker
directory_fin_var="Financial_values/"

for name in stocknames["ticker"].values:
    file_path = os.path.join(directory_fin_var, name)
    if not os.path.exists(file_path):
            os.makedirs(file_path)
            pass

In [5]:
#connect to WRDS
db = wrds.Connection(wrds_username = 'nmonnet')
db.create_pgpass_file()

Loading library list...
Done


Enter your WRDS username [nmonnet]: nmonnet
Enter your password: ········


In [6]:
series_list=[]
for i in tqdm(range(len(stocknames)),desc = 'Getting daily returns for all firms', leave = False):
    no_error=True
    for nb_attempt in range(10):
        try:
            permco=stocknames.permco[i]
            start_date=stocknames.st_date[i]

            if int(start_date[:4])<2011:
                start_date="2011-01-01"
            


            
            end_date=stocknames.end_date[i]
            tick=stocknames.ticker[i]
            Request = """select date, ret from crsp.dsf where permco in ({})
                          and date >= '{}' and date <='{}'""".format(permco, start_date, end_date)
            data = db.raw_sql(Request)
        
            data.set_index('date', inplace = True)
            data.index = pd.to_datetime(data.index)
            table=data.sort_index()

            
            # Values to drop
            values_to_drop = [-66, -77, -88, -99]

            # Drop rows containing specific values
            table = table[~table['ret'].isin(values_to_drop)]

            table = table[~table.index.duplicated(keep='first')]

            table.rename(columns={'ret': tick}, inplace=True)
        
            series_list.append(table)
       
        except:
            no_error=False
            
        if no_error:
            break
    if not(no_error):
        print("There was a problem that could not be resolved in 10 tries")
        print(i)
    #break #to be removed

Getting daily returns for all firms:   0%|          | 0/7083 [00:00<?, ?it/s]

In [7]:
returns_table=pd.concat(series_list, axis=1)
returns_table.sort_index(inplace=True)

In [8]:
returns_table

,TSLA,HBIO,BELFA,AMPX,RENT,INLX,NTRS,PYXS,SPGI,XPER,...,FLYD,BCRD,MSPC,MSTC,HRST,GWIN,CLCN,BRVO,BRER,ATDS
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,-0.000375,0.000000,0.025523,NaN,NaN,NaN,0.009385,NaN,0.004394,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,0.001878,-0.004902,-0.006971,NaN,NaN,NaN,0.001252,NaN,0.007383,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,0.005999,-0.009852,0.011998,NaN,NaN,NaN,0.009821,NaN,0.017915,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,0.039135,0.037314,-0.016762,NaN,NaN,NaN,-0.003537,NaN,0.009333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-07,0.012913,0.004796,0.003742,NaN,NaN,NaN,-0.019343,NaN,-0.002906,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,-0.007701,0.022727,0.036537,0.010695,-0.012054,0.039920,0.000477,0.008982,0.004969,0.000937,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26,0.016116,-0.001852,0.025195,0.066138,-0.016218,0.029770,0.004890,0.032641,0.003197,0.037453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-27,0.018822,0.018553,0.004048,0.188586,-0.081065,-0.023317,0.003798,0.000000,0.008642,-0.002708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
save_path='Financial_ratios/'
returns_table.to_csv(save_path + 'returns_daily.csv.gz')

In [ ]:
STOP

In [3]:
#test
file_path = 'Financial_ratios/returns_daily.csv.gz'
returns_table=pd.read_csv(file_path)
returns_table.set_index('date', inplace=True)

In [4]:
returns_table

,TSLA,HBIO,BELFA,AMPX,RENT,INLX,NTRS,PYXS,SPGI,XPER,...,FLYD,BCRD,MSPC,MSTC,HRST,GWIN,CLCN,BRVO,BRER,ATDS
date,,,,,,,,,,,,,,,,,,,,,
2011-01-03,-0.000375,0.000000,0.025523,NaN,NaN,NaN,0.009385,NaN,0.004394,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,0.001878,-0.004902,-0.006971,NaN,NaN,NaN,0.001252,NaN,0.007383,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-05,0.005999,-0.009852,0.011998,NaN,NaN,NaN,0.009821,NaN,0.017915,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-06,0.039135,0.037314,-0.016762,NaN,NaN,NaN,-0.003537,NaN,0.009333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-07,0.012913,0.004796,0.003742,NaN,NaN,NaN,-0.019343,NaN,-0.002906,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,-0.007701,0.022727,0.036537,0.010695,-0.012054,0.039920,0.000477,0.008982,0.004969,0.000937,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26,0.016116,-0.001852,0.025195,0.066138,-0.016218,0.029770,0.004890,0.032641,0.003197,0.037453,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-27,0.018822,0.018553,0.004048,0.188586,-0.081065,-0.023317,0.003798,0.000000,0.008642,-0.002708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
